In [1]:
from data_processor_lib import Processor

data_src = "https://media.geeksforgeeks.org/wp-content/uploads/nba.csv"

In [2]:
processor = Processor()
processor.read_csv(data_src)

Profiler initialized


In [5]:
processor.columns()

Index(['Name', 'Team', 'Number', 'Position', 'Age', 'Height', 'Weight',
       'College', 'Salary'],
      dtype='object')

In [6]:
processor.set_ignore(['Name', 'College'])

In [8]:
processor.overview()

,Value
Total Columns,458.0
Total Rows,7.0
Incomplete Rows Count,NaN
Incomplete Rows (%),NaN
Incomplete Columns Count,NaN
Total Data Points,NaN
Incomplete Data Points Count,NaN
Incomplete Data (%),NaN
